In [84]:
import json
import re

In [85]:
with open('datasets/arc_challenge_train_gpt4_response.json', 'r') as f:
    data = json.loads(f.read())

In [86]:
len(data)

1000

In [87]:
print(data['Mercury_7126683']['response'])

```json
{
  "concepts": [
    {
      "name": "Carbon Cycle",
      "description": "The series of processes by which carbon compounds are interconverted in the environment."
    },
    {
      "name": "Atmosphere",
      "description": "The layer of gases surrounding Earth."
    },
    {
      "name": "Formation of Fossil Fuels",
      "description": "The process by which decomposed organic material is converted under pressure and heat into coal, oil, and natural gas over millions of years."
    },
    {
      "name": "Layering of Soil",
      "description": "The formation of distinct layers within the soil, each with its own characteristics."
    },
    {
      "name": "Plant Photosynthesis",
      "description": "The process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water."
    },
    {
      "name": "Forest Fires",
      "description": "Uncontrolled fire occurring in nature."
    }
  ],
  "relationships": [
    {
      "c

In [88]:
# Function to extract all markdown json tags content from response
def extract_json_content(response):
    # Pattern to match json content within markdown code blocks
    pattern = r'```json\n(.*?)\n```'
    # Using re.DOTALL to make '.' match any character including newline
    matches = re.findall(pattern, response, re.DOTALL)
    return matches

In [89]:
# Example usage
response = data['Mercury_7126683']['response']
json_contents = extract_json_content(response)
for json_content in json_contents:
    print(json_content)

{
  "concepts": [
    {
      "name": "Carbon Cycle",
      "description": "The series of processes by which carbon compounds are interconverted in the environment."
    },
    {
      "name": "Atmosphere",
      "description": "The layer of gases surrounding Earth."
    },
    {
      "name": "Formation of Fossil Fuels",
      "description": "The process by which decomposed organic material is converted under pressure and heat into coal, oil, and natural gas over millions of years."
    },
    {
      "name": "Layering of Soil",
      "description": "The formation of distinct layers within the soil, each with its own characteristics."
    },
    {
      "name": "Plant Photosynthesis",
      "description": "The process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water."
    },
    {
      "name": "Forest Fires",
      "description": "Uncontrolled fire occurring in nature."
    }
  ],
  "relationships": [
    {
      "concept":

In [90]:
concept_missing_count = 0
rel_missing_count = 0
final_answer_missing_count = 0
missing_final_answer_keys = []
for key, val in data.items():
    response = val['response']
    json_contents = extract_json_content(response)
    parsed_response = {}
    for content in json_contents:
        try:
            json_object = json.loads(content)
        except Exception as e:
            print(key)
            continue
        if 'concepts' in json_object:
            parsed_response['concepts'] = json_object['concepts']
        if 'relationships' in json_object:
            parsed_response['relationships'] = json_object['relationships']
        if 'answer' in json_object:
            if 'reasoning' in json_object:
                parsed_response['final_answer'] = json_object
            else:
                parsed_response['final_answer'] = json_object['answer']
    if 'concepts' not in parsed_response:
        concept_missing_count += 1
    if 'relationships' not in parsed_response:
        rel_missing_count += 1
    if 'final_answer' not in parsed_response:
        final_answer_missing_count += 1
        missing_final_answer_keys.append(key)
    data[key]['parsed_response'] = parsed_response

In [92]:
final_answer_missing_count, concept_missing_count, rel_missing_count

(0, 0, 0)

In [93]:
missing_final_answer_keys

[]

In [94]:
ARC_PARSED_DATASET_PATH = 'datasets/arc_challenge_train_gpt4_response_parsed.json'

In [95]:
with open(ARC_PARSED_DATASET_PATH, 'w') as file:
	json.dump(data, file)